In [ ]:
!pip install neo4j

In [ ]:
from neo4j import GraphDatabase

# Create a new Driver instance
driver = GraphDatabase.driver("neo4j://localhost:7687",auth=("neo4j", "neo"))

In [ ]:
# Verify the connection details
driver.verify_connectivity()

# Sessions

In [ ]:
with driver.session(database="people") as session:

session.run(
    "MATCH (p:Person {name: $name}) RETURN p", # Query
    name="Tom Hanks" # Named parameters referenced
)         

# Transactions

## execute_read

In [ ]:
def get_movies(tx, title):
    return tx.run("""
        MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
        WHERE m.title = $title // (1)
        RETURN p.name AS name
        LIMIT 10
    """, title=title)

# Execute get_movies within a Read Transaction
session.execute_read(get_movies,
    title="Arthur" # (2)
)

## execute_write

def create_person(tx, name):
    return tx.run(
        "CREATE (p:Person {name: $name})",
        name=name
    )


# Execute the `create_person` "unit of work" within a write transaction
session.execute_write(create_person, name="Michael")

# Manually Creating Transactions

In [ ]:
with session.begin_transaction() as tx:

try:
    # Run a query
    tx.run(query, **params)

    # Commit the transaction
    tx.commit()
except:
    # If something goes wrong in the try block,
    # then rollback the transaction
    tx.rollback()

# Closing the Session

In [ ]:
session.close()